In [3]:
import numpy as np
import hdf5storage
import time
from scipy import signal
from scipy.signal import butter, lfilter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs 

def Epoching(eegData, stims, samplingFreq, channelNum, epochSampleNum, offset, baseline):
        Time_after = np.add(stims,offset).astype(int)
        Time_base = np.add(stims,baseline).astype(int)
        Num = stims.shape[1]
        Epochs = np.zeros((Num, channelNum, epochSampleNum))
        for j in range(Num):
            Epochs[j, :, :] = eegData[:,Time_after[0][j]:Time_after[0][j] + epochSampleNum]
            
            #Baseline Correction
            for i in range(Epochs.shape[1]):
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                
        return [Epochs,Num]

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
    EpochsT = Standardization(EpochsT)
    EpochsN = Standardization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)

    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def resampling(Epochs, EpochNum, resampleRate, channelNum):
        resampled_epoch = np.zeros((EpochNum, channelNum, resampleRate))
        for i in range(EpochNum):
            for j in range(channelNum):
                resampled_epoch[i,j,:] = signal.resample(Epochs[i,j,:], resampleRate)
                
        return resampled_epoch
    
def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]
    
def main():
    start = time.time()
    
    root_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\P300Biosemi55\\'
    subject = 'S04'
    
    filename = root_path + subject
    
    Classifier_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\LDAModel\\New\\' + subject + 'Classifier.pickle'
    
    channelNum = 32
    resampleRate = 128
    
    target = np.zeros((300,channelNum,resampleRate))
    nontarget = np.zeros((300,channelNum,resampleRate))
    for i in range(1,3):
        if i == 2:
            filename = filename + '_2'
            
        mat = hdf5storage.loadmat(filename)
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq'][0,0]
        stimsN = mat['stimsN']
        stimsT = mat['stimsT']
        sampleNum = eegData.shape[1]
        
        ## Preprocessing process
        eegData = Re_referencing(eegData, channelNum, eegData.shape[1])
            
        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, 4)
        
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq))
        baseline = int(np.floor(1.0 * samplingFreq))
        [EpochsT, NumT] = Epoching(eegData, stimsT, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stimsN, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        #Data Balancing
        EpochsN_New = np.zeros((NumT, channelNum, epochSampleNum))
        NumN = NumT
        for j in range(NumN):
            EpochsN_New[j,:,:] = np.mean(EpochsN[j*5:j*5+5, :, :], axis=0)
        
        #Standardization
#         EpochsT = Standardization(EpochsT)
#         EpochsN = Standardization(EpochsN)
        
#         [EpochsT, NumT, EpochsN, NumN] = Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, 20)
        
        #Resampling
        EpochsT = resampling(EpochsT, NumT, resampleRate, channelNum) 
        EpochsN = resampling(EpochsN_New, NumN, resampleRate, channelNum)
        
        target[150*(i-1):150*i,:,:] = EpochsT
        nontarget[150*(i-1):150*i,:,:] = EpochsN
    
    #Convert to feature vector
    featureNum = channelNum*resampleRate
    [FeaturesT, FeaturesN] = Convert_to_featureVector(target, 300, nontarget, 300, featureNum)
    Data = np.concatenate((FeaturesT, FeaturesN))
    Label = np.concatenate((np.ones((300,1)).astype(int),np.zeros((300,1)).astype(int))).ravel()
    
    s = np.arange(Data.shape[0])
    np.random.shuffle(s)
    
    Data = Data[s]
    Label = Label[s]
    
    TrainData = Data[0:480]
    TrainLabel = Label[0:480]
    
    TestData = Data[480:]
    TestLabel = Label[480:]
    
    #Saving LDA classifier
    lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
    lda.fit(TrainData, TrainLabel)
    joblib.dump(lda, Classifier_path, protocol=2)
    
    print('Done')
    print("time :", time.time() - start)
    
    model = joblib.load(Classifier_path)
    
    result = model.decision_function(TestData)
    predict = model.predict(TestData)
    
    print(result)
    print(predict)
    
    decision_score = 0
    predict_score = 0
    for i in range(len(result)):
        if result[i] > 0:
            decision = 1
        else:
            decision = 0
        
        if TestLabel[i] == decision:
            decision_score = decision_score + 1
        if TestLabel[i] == predict[i]:
            predict_score = predict_score + 1
    
    print('decision:', decision_score, '/', len(result))
    print('predict:', predict_score, '/', len(result))
    
if __name__ == "__main__":
    main()

Done
time : 30.408833980560303
[-52.73199735 -18.29860821 -37.56974776  47.28853804 -25.480341
  57.10707443 -26.43792758 -29.58603222 -33.5583033   31.37090672
 -33.56597368  16.77329333  11.4543571  -14.2307663  -14.05465346
  -3.55851621   1.40894892  45.34487733 -27.71017425 -34.15921669
  14.28234878  61.40979352 -16.84897241 -42.08427656 -15.10151351
  16.59806528  54.12236856  44.56335596 -32.67759703   6.58267687
 -11.01887121  22.08524232 -36.28261673 -36.32793009  33.43797453
 -28.01609401 -24.8202386  -29.86331869   9.47299716  20.39660808
  17.80815069 -17.48436702  -9.83911172 -62.84120169 -12.02542721
  48.61013887  15.09107959 -15.04817045 -14.45336507 -21.90851913
 -29.41781579  52.78302454 -26.22076784 -19.73342995 -23.5513434
 -23.24516696 -18.47719111 -13.12925108 -14.12140943 -24.85064302
 -39.93938642  -9.5289143   -2.06342815  -5.00636415  11.01086195
 -12.43438876 -25.37322934  -7.86288548  -7.76085027 -28.87547886
 -27.64500342 -21.52307485  38.30995972 -20.2845

In [4]:
import numpy as np
import hdf5storage
import time
from scipy import signal
from scipy.signal import butter, lfilter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib

def Re_referencing(eegData, channelNum, sampleNum):
        after_car = np.zeros((channelNum,sampleNum))
        for i in np.arange(channelNum):
            after_car[i,:] = eegData[i,:] - np.mean(eegData,axis=0)
        return after_car

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs 

def Epoching(eegData, stims, samplingFreq, channelNum, epochSampleNum, offset, baseline):
        Time_after = np.add(stims,offset).astype(int)
        Time_base = np.add(stims,baseline).astype(int)
        Num = stims.shape[1]
        Epochs = np.zeros((Num, channelNum, epochSampleNum))
        for j in range(Num):
            Epochs[j, :, :] = eegData[:,Time_after[0][j]:Time_after[0][j] + epochSampleNum]
            
            #Baseline Correction
            for i in range(Epochs.shape[1]):
                Epochs[j,i,:] = Epochs[j,i,:] - np.mean(Epochs[j,i,:])
                
        return [Epochs,Num]

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
    EpochsT = Standardization(EpochsT)
    EpochsN = Standardization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)

    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]

def resampling(Epochs, EpochNum, resampleRate, channelNum):
        resampled_epoch = np.zeros((EpochNum, channelNum, resampleRate))
        for i in range(EpochNum):
            for j in range(channelNum):
                resampled_epoch[i,j,:] = signal.resample(Epochs[i,j,:], resampleRate)
                
        return resampled_epoch
    
def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]
    
def main():
    start = time.time()
    
    root_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\EEGData\\P300Biosemi55\\'
    subject = 'S04'
    
    filename = root_path + subject
    
    Classifier_path = 'C:\\Users\\hyuns\\Desktop\\2020-2\\캡스톤\\LDAModel\\New\\' + subject + 'Classifier.pickle'
    
    channelNum = 32
    resampleRate = 128
    
    target = np.zeros((300,channelNum,resampleRate))
    nontarget = np.zeros((1500,channelNum,resampleRate))
    for i in range(1,3):
        if i == 2:
            filename = filename + '_2'
            
        mat = hdf5storage.loadmat(filename)
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq'][0,0]
        stimsN = mat['stimsN']
        stimsT = mat['stimsT']
        sampleNum = eegData.shape[1]
        
        ## Preprocessing process
        eegData = Re_referencing(eegData, channelNum, eegData.shape[1])
            
        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, 4)
        
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq))
        baseline = int(np.floor(1.0 * samplingFreq))
        [EpochsT, NumT] = Epoching(eegData, stimsT, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stimsN, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        #Data Balancing
#         EpochsN_New = np.zeros((NumT, channelNum, epochSampleNum))
#         NumN = NumT
#         for j in range(NumN):
#             EpochsN_New[j,:,:] = np.mean(EpochsN[j*5:j*5+5, :, :], axis=0)
        
        #Standardization
#         EpochsT = Standardization(EpochsT)
#         EpochsN = Standardization(EpochsN)
        
#         [EpochsT, NumT, EpochsN, NumN] = Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, 20)
        
        #Resampling
        EpochsT = resampling(EpochsT, NumT, resampleRate, channelNum) 
        EpochsN = resampling(EpochsN, NumN, resampleRate, channelNum)
        
        target[150*(i-1):150*i,:,:] = EpochsT
        nontarget[750*(i-1):750*i,:,:] = EpochsN
    
    #Convert to feature vector
    featureNum = channelNum*resampleRate
    [FeaturesT, FeaturesN] = Convert_to_featureVector(target, 300, nontarget, 1500, featureNum)
    Data = np.concatenate((FeaturesT, FeaturesN))
    Label = np.concatenate((np.ones((300,1)).astype(int),np.zeros((1500,1)).astype(int))).ravel()
    
    s = np.arange(Data.shape[0])
    np.random.shuffle(s)
    
    Data = Data[s]
    Label = Label[s]
    
    TrainData = Data[0:1500]
    TrainLabel = Label[0:1500]
    
    TestData = Data[1500:]
    TestLabel = Label[1500:]
    
    #Saving LDA classifier
    lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
    lda.fit(TrainData, TrainLabel)
    joblib.dump(lda, Classifier_path, protocol=2)
    
    print('Done')
    print("time :", time.time() - start)
    
    model = joblib.load(Classifier_path)
    
    result = model.decision_function(TestData)
    predict = model.predict(TestData)
    
    print(result)
    print(predict)
    
    decision_score = 0
    predict_score = 0
    for i in range(len(result)):
        if result[i] > 0:
            decision = 1
        else:
            decision = 0
        
        if TestLabel[i] == decision:
            decision_score = decision_score + 1
        if TestLabel[i] == predict[i]:
            predict_score = predict_score + 1
    
    print('decision:', decision_score, '/', len(result))
    print('predict:', predict_score, '/', len(result))
    
if __name__ == "__main__":
    main()

Done
time : 35.34953594207764
[ -2.8882325    8.79668112   1.87255926  -9.10893448  -9.41961455
   2.06972477  -7.62705075 -11.7084387    4.28734787  -9.57924043
 -12.08618098  -9.82210188 -14.51469236  -5.65117138  -7.17003216
 -14.17953053 -16.95264569  -5.98042123  -9.09317976  -9.03170054
  -1.51588648  -5.55104514   1.8544442  -22.02439777 -11.51498785
  -0.04518944 -17.73707467  -6.71602523  -0.04818083 -11.60121608
  -1.40447191 -10.89161105  -8.07579531  -8.01901887  -7.28646233
   1.7868999   -6.75682023 -12.75753434  -7.16196504 -11.71851666
 -10.40388066  -5.69376977  19.49977179  -6.8670501   -3.87003188
 -16.13005392   7.61095284  -6.72806659   4.3449421   -4.95872102
  -1.05523187   1.2876197  -21.6244926   -2.00234262   3.74768456
  -1.03349879  -1.9475968    0.99803765   4.0389946  -25.31238049
  -3.17362106  -3.16666587 -20.48255581 -10.40993066 -14.89762286
  -8.17505177  -9.01077131  -2.14530682  -8.39471439  -3.40111617
  -5.27214395 -10.35361325  -0.88910275  -5.36